<a href="https://colab.research.google.com/github/capozzolicaluan/MSWEPtest/blob/main/error_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Limpar todas as variáveis
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [6]:
!pip install HydroErr -q

  Preparing metadata (setup.py) ... done


In [7]:
# Importando as bibliotecas necessárias
import requests
import pandas as pd
import gc
from google.colab import data_table
import math
import HydroErr as he

# Liberar memória manualmente
_ = gc.collect()

In [69]:
import numpy as np

def cont_index(obs, sim, limiar=0.5):
    """
    Calcula os índices POD, FAR e CSI a partir de séries observadas e previstas de chuva.

    Parâmetros:
    - hidro_sem_nan: Série pandas (ou array NumPy) com dados observados (sem NaN).
    - mswep_sem_nan: Série pandas (ou array NumPy) com dados previstos (sem NaN).
    - limiar: Valor em mm para considerar a ocorrência de chuva (default: 0.5 mm).

    Retorna:
    - dicionário com hits, misses, false_alarms, correct_negatives, pod, far e csi.
    """

    # Converte para arrays NumPy, se forem séries pandas
    observado = np.asarray(obs)
    previsto = np.asarray(sim)

    # BIAS relativo
    y_true = observado.reshape(len(observado),1)
    y_pred = previsto.reshape(len(previsto),1)
    diff = (y_true-y_pred)
    diff_sum = diff.sum()
    y_true_sum = y_true.sum()
    rbias = diff_sum/y_true_sum

    # Aplica o limiar para binarizar as observações e previsões
    observado_chuva = observado >= limiar
    previsto_chuva = previsto >= limiar



    # Tabela de contingência
    hits = ((observado_chuva == True) & (previsto_chuva == True)).sum()
    misses = ((observado_chuva == True) & (previsto_chuva == False)).sum()
    false_alarms = ((observado_chuva == False) & (previsto_chuva == True)).sum()
    correct_negatives = ((observado_chuva == False) & (previsto_chuva == False)).sum()

    # Cálculo dos índices com verificação para evitar divisão por zero
    pod = hits / (hits + misses) if (hits + misses) > 0 else np.nan
    far = false_alarms / (false_alarms + correct_negatives) if (false_alarms + correct_negatives) > 0 else np.nan
    csi = (pod - far) / (1 - far) if (1 - far) != 0 else np.nan

    return {
        'rbias': rbias,
        'pod': pod,
        'far': far,
        'csi': csi
    }


def filt_month(df, month):
  # Certifique-se de que a coluna 'datetime' está no formato datetime
  df['datetime'] = pd.to_datetime(df['datetime'])

  # Para filtrar, por exemplo, somente dados de janeiro (mês 1)
  df_mes = df[df['datetime'].dt.month == month]

  return (df_mes)

def remove_nan(df_obs, df_sim):

  sem_nan = df_obs.dropna().index
  obs_sem_nan = df_obs.loc[sem_nan]
  sim_sem_nan = df_sim.loc[sem_nan]

  return obs_sem_nan, sim_sem_nan

def salva_indices(df_indice, caminho_salvar, nome_indice):

  if isinstance(df_indice, pd.core.frame.DataFrame):
    pd_df_indice = df_indice
  else:
    pd_df_indice = pd.DataFrame({'Indice': df_indice})

  pd_df_indice.to_csv(join(caminho_salvar,nome_indice + ".txt"), sep=';', index=False)

  print(nome_indice)


def lista_saidas():

  # Lista para armazenar variáveis que começam com "erro_"
  variaveis_erro = []

  # Itera sobre as variáveis globais
  for var_name, var_value in list(globals().items()):
    # Verifica se o nome da variável começa com "erro_"
    if var_name.startswith("erro_"):
      variaveis_erro.append(var_name)

  return variaveis_erro

In [9]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [10]:
from posixpath import join
file_path = "/content/drive/MyDrive/Colab/txts"

dados_hidro = pd.read_csv(join(file_path, "matriz_hidro.txt"), sep = ';')
dados_mswep = pd.read_csv(join(file_path, "matriz_mswep.txt"), sep = ';')

In [11]:
for i in range(dados_mswep.shape[1]-1):
  indices = dados_mswep[ dados_mswep.iloc[:,(i+1)] < 0.2].index
  dados_mswep.iloc[indices,(i+1)] = 0

In [12]:
for i in range(dados_hidro.shape[1]-1):
  indices = dados_hidro[ dados_hidro.iloc[:,(i+1)] < 0.2].index
  dados_hidro.iloc[indices,(i+1)] = 0

In [13]:
# statistics metrics: correlation coef, relative bias, rmse, pod, false alarm ratio, critical success index
erro_cc = [None] * (dados_mswep.shape[1]-1)
erro_bias = [None] * (dados_mswep.shape[1]-1)
erro_rmse = [None] * (dados_mswep.shape[1]-1)
erro_far = [None] * (dados_mswep.shape[1]-1)
erro_pod = [None] * (dados_mswep.shape[1]-1)
erro_csi = [None] * (dados_mswep.shape[1]-1)

for estacao_i in range(dados_mswep.shape[1]-1):

  hidro_sem_nan, mswep_sem_nan = remove_nan(df_obs = dados_hidro.iloc[:, estacao_i+1],
                                            df_sim = dados_mswep.iloc[:, estacao_i+1])

  erro_cc[estacao_i] = he.pearson_r(simulated_array=mswep_sem_nan, observed_array=hidro_sem_nan)
  erro_rmse[estacao_i] = he.rmse(simulated_array=mswep_sem_nan, observed_array=hidro_sem_nan)
  erro_bias[estacao_i] = cont_index(obs=hidro_sem_nan, sim=mswep_sem_nan)['rbias']
  erro_far[estacao_i] = cont_index(obs=hidro_sem_nan, sim=mswep_sem_nan)['far']
  erro_pod[estacao_i] = cont_index(obs=hidro_sem_nan, sim=mswep_sem_nan)['pod']
  erro_csi[estacao_i] = cont_index(obs=hidro_sem_nan, sim=mswep_sem_nan)['csi']

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/HydroErr/HydroErr.py:1534: RuntimeWarning: invalid value encountered in scalar divide
  return top / (bot1 * bot2)
/tmp/ipython-input-8-1004498154.py:26: RuntimeWarning: invalid value encountered in scalar divide
  rbias = diff_sum/y_true_sum


In [16]:
# statistics metrics: correlation coef, relative bias, rmse, pod, false alarm ratio, critical success index
erro_cc_mes = pd.DataFrame([[None]*12 for _ in range( (dados_mswep.shape[1]-1))])
erro_bias_mes = erro_cc_mes.copy()
erro_rmse_mes = erro_cc_mes.copy()
erro_far_mes = erro_cc_mes.copy()
erro_pod_mes = erro_cc_mes.copy()
erro_csi_mes = erro_cc_mes.copy()

for mes_m in range(0,12):

  df_mes_hidro = filt_month(df = dados_hidro, month = mes_m+1)
  df_mes_mswep = filt_month(df = dados_mswep, month = mes_m+1)

  for estacao_i in range(dados_mswep.shape[1]-1):

    hidro_sem_nan, mswep_sem_nan = remove_nan(df_obs = df_mes_hidro.iloc[:, estacao_i+1],
                                              df_sim = df_mes_mswep.iloc[:, estacao_i+1])

    erro_cc_mes.at[estacao_i,mes_m] = he.pearson_r(simulated_array=mswep_sem_nan, observed_array=hidro_sem_nan)
    erro_rmse_mes.at[estacao_i,mes_m] = he.rmse(simulated_array=mswep_sem_nan, observed_array=hidro_sem_nan)
    erro_bias_mes.at[estacao_i,mes_m] = cont_index(obs=hidro_sem_nan, sim=mswep_sem_nan)['rbias']
    erro_far_mes.at[estacao_i,mes_m] = cont_index(obs=hidro_sem_nan, sim=mswep_sem_nan)['far']
    erro_pod_mes.at[estacao_i,mes_m] = cont_index(obs=hidro_sem_nan, sim=mswep_sem_nan)['pod']
    erro_csi_mes.at[estacao_i,mes_m] = cont_index(obs=hidro_sem_nan, sim=mswep_sem_nan)['csi']

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/HydroErr/HydroErr.py:1534: RuntimeWarning: invalid value encountered in scalar divide
  return top / (bot1 * bot2)
/tmp/ipython-input-8-1004498154.py:26: RuntimeWarning: invalid value encountered in scalar divide
  rbias = diff_sum/y_true_sum
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/HydroErr/HydroErr.py:1534

In [51]:
saidas = lista_saidas()

In [70]:
for saida in saidas:
    salva_indices(df_indice = globals()[saida], caminho_salvar = '/content/drive/MyDrive/Colab/txts', nome_indice = saida)

erro_cc
erro_bias
erro_rmse
erro_far
erro_pod
erro_csi
erro_cc_mes
erro_bias_mes
erro_rmse_mes
erro_far_mes
erro_pod_mes
erro_csi_mes


In [45]:
caminho_salvar = '/content/drive/MyDrive/Colab/txts'
nome_indice = saida
df_indice = globals()[saida]

In [ ]:
# prompt: concatenar caminho_salvar, nome_indice, df_indice e '.txt'

file_path_complete = join(caminho_salvar, nome_indice + '.txt')
file_path_complete

In [48]:
join(caminho_salvar,nome_indice + ".txt")

'/content/drive/MyDrive/Colab/txts/erro_cc.txt'

In [ ]:
join(caminho_salvar,nome_indice,".txt")

In [ ]:
pd_df_indice.to_csv(join(caminho_salvar,nome_indice,".txt"), sep=';', index=False)

In [66]:
tt = pd.DataFrame({'Indice': erro_cc})

In [ ]:
# prompt: conferir se uma variavel é pandas.core.frame.DataFrame

import pandas as pd

print(isinstance(tt, pd.core.frame.DataFrame))


In [68]:
isinstance(tt, pd.core.frame.DataFrame)

True